In [1]:
import pandas as pd
import numpy as np
import datetime
import pickle
import tensorflow as tf

2023-03-06 00:29:48.438641: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 00:29:50.026649: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /hpc/mp/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-10.3.0/cudnn-8.2.4.15-11.4-eluwegpwn6adr7hlku5p5wru5xzefpop/lib64:/hpc/mp/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-10.3.0/cuda-11.4.4-ctldo35wmmwws3jbgwkgjjcjawddu3qz/lib64:/hpc/mp/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-10.3.0/neovim-0.7.0-terkir3wk5rst6ktv4uxyaqjditacv5p/lib
2023-03-06 00:29:50.026741: W tensorflow/compiler/xla/stream_execut

In [2]:
# make tensorlfow work better with gpus:
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

# eager execution:
tf.compat.v1.enable_eager_execution(
    config=None, device_policy=None, execution_mode=None
)

# Overview of Dataset

What is the classification task and what is the format of the feature data. Is this multi-task, multi-modal, or both? Explain.
Who collected the data? Why? When?
What evaluation criteria will you be using and why? 

https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/

We are using the IMDB-Wiki dataset for multi-task learning of Age and Gender from facial image data. We will be doing multi-task by training a model to classify geneder and regress age of a certain person based on their picture.


This data was collecteted by 


In [3]:
# CONSTANTS

DATA_DIR = "/work/users/skoka/Data/wiki_crop/"
META_DATA =  "/work/users/skoka/Data/wiki_crop/wiki.mat"

PICKLES_PATH = "/work/users/skoka/Data/wiki_crop_pickles/"
GET_IMAGES_FROM_PICKLE = True

In [4]:
# extract data from a .mat file
import scipy.io as sio


mat = sio.loadmat(META_DATA)
# convert the data to a pandas dataframe
mat_data = mat['wiki'][0][0]

# 0 -> Matlab datatime format
# 1 -> Years
# 2 -> Path to image
# 3 -> Gender

# convert Matlab serial date number to datetime object
def matlab_to_datetime(matlab_serial_date):
    python_datetime = datetime.datetime.fromordinal(int(matlab_serial_date)) + \
                      datetime.timedelta(days=matlab_serial_date % 1) - \
                      datetime.timedelta(days=366)
    return python_datetime

def get_age(born_matlab_date, current_year):
    return current_year - matlab_to_datetime(born_matlab_date).year

new_data = [
    # {
        # image_path, gender, age
    # }
]

for i in range(len(mat_data[0][0])):
    new_data.append({
        "image_path": mat_data[2][0][i][0],
        "age": get_age(int(mat_data[0][0][i]), mat_data[1][0][i]),
        "gender": mat_data[3][0][i]
    })

df = pd.DataFrame(new_data)

In [5]:
# reading in images:
from PIL import Image
import os

def read_image(path):
    image = Image.open(os.path.join(DATA_DIR, path))
    image = image.resize((256, 256))
    return np.array(image)

if not GET_IMAGES_FROM_PICKLE:
    list_of_images = []
    for i in range(len(df)):
        list_of_images.append(read_image(df["image_path"][i]))
    with open(PICKLES_PATH + "list_of_images.pickle", "wb") as f:
        pickle.dump(list_of_images, f)
else:
    with open(PICKLES_PATH + "list_of_images.pickle", "rb") as f:
        list_of_images = pickle.load(f)

In [6]:
# making every image 3 channels if it isnt already
for i in range(len(list_of_images)):
    if len(list_of_images[i].shape) != 3:
        list_of_images[i] = np.stack((list_of_images[i],) * 3, axis=-1)

# adding the images to the dataframe
df["image"] = list_of_images
# drop na from the dataframe
df = df.dropna()

In [7]:
X = np.array(df["image"])
X = np.concatenate(X)
X = X.reshape(-1, 256, 256, 3)
X = X / 255.0
Y_age = df["age"].values.astype('float32')
Y_gender = df["gender"].values.astype('float32')

In [8]:
X.shape

(59685, 256, 256, 3)

In [9]:
# Train test spliut:
from sklearn.model_selection import train_test_split


# Seperate Models

## Gender Classification:

In [10]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Dropout, BatchNormalization, Activation
from keras.models import Model
import tensorflow 

# strategy = tf.distribute.MirroredStrategy(
#     devices=["/gpu:0", "/gpu:1", "/gpu:2", "/gpu:3", "/gpu:4", "/gpu:5", "/gpu:6", "/gpu:7"],
#     cross_device_ops=tf.distribute.HierarchicalCopyAllReduce()
# )

# with strategy.scope():

input_layer = Input(shape=(256, 256, 3))
CNN_1 = Conv2D(32, (3, 3), padding='same')(input_layer)
CNN_1 = BatchNormalization()(CNN_1)
CNN_2 = Conv2D(32, (3, 3), padding='same')(CNN_1)
CNN_2 = BatchNormalization()(CNN_2)
CNN_2 = Activation('relu')(CNN_2)
CNN_2 = MaxPooling2D((2, 2), padding='same')(CNN_2)
CNN_2 = Dropout(0.25)(CNN_2)
CNN_3 = Conv2D(64, (3, 3), padding='same')(CNN_2)
CNN_3 = BatchNormalization()(CNN_3)

dense = Flatten()(CNN_3)
output = Dense(1)(dense)

gender_classification_model = Model(input_layer, output)

gender_classification_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

2023-03-06 00:30:19.380475: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 00:30:23.595557: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79117 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2023-03-06 00:30:23.599522: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 79117 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:0f:00.0, compute capability: 8.0
2023-03-06 00:30:23.603183: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/devi

In [11]:
class_weights = {
        0: 80,
        1: 20
    }


gender_classification_model.fit(X,
                                Y_gender,
                                epochs=100,
                                batch_size=256,
                                class_weight=class_weights,
                                callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=2, min_lr=0.00000001, verbose=1)])

Epoch 1/100


2023-03-06 00:31:05.851033: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-03-06 00:31:08.308310: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8204
2023-03-06 00:31:10.807325: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-06 00:31:10.808379: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fc105854aa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-06 00:31:10.808399: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2023-03-06 00:31:10.808403: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecut